## Session Setup

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark aggregation functions") \
    .getOrCreate()

### Loading Data

In [2]:
listings = spark.read.csv("../data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: timestamp (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: timestamp (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_

In [3]:
reviews = spark.read.csv("../data/reviews.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)
reviews.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [4]:
# 1. Count the number of reviews per listing using the "reviews" dataset

reviews_per_listing = reviews \
  .groupBy('listing_id') \
  .count() \
  .show(10)

+----------+-----+
|listing_id|count|
+----------+-----+
|     78606|    2|
|    444886|   12|
|    466017|   28|
|   2736493|    4|
|   2557853|   89|
|   3132302|    3|
|   3917692|    1|
|   3734796|    5|
|   3997029|    7|
|   4361078|   70|
+----------+-----+
only showing top 10 rows



In [5]:
# 2. Compute the total number of listings and average review score per host

from pyspark.sql.functions import avg, count

host_stats = listings \
  .filter(listings.review_scores_rating.isNotNull()) \
  .groupBy('host_id') \
  .agg(
    count('id').alias('total_listings'),
    avg('review_scores_rating').alias('average_review_score')
  ) \
  .show(10)

+--------+--------------+--------------------+
| host_id|total_listings|average_review_score|
+--------+--------------+--------------------+
| 2358441|             1|                4.86|
| 2876123|             2|  4.9399999999999995|
| 2038199|             1|                 5.0|
| 4157822|             2|               4.925|
|  719504|             1|                4.96|
| 7950720|             1|                4.86|
| 6572018|             1|                 5.0|
|12122942|             1|                4.93|
|13851928|             1|                4.97|
|13739634|             2|                4.74|
+--------+--------------+--------------------+
only showing top 10 rows



In [6]:
# 3: Find the top ten listings with the highest number of reviews

reviews \
  .groupBy('listing_id') \
  .count() \
  .orderBy('count', ascending=False) \
  .limit(10) \
  .show()

+----------+-----+
|listing_id|count|
+----------+-----+
|  47408549| 1855|
|  30760930| 1682|
|  43120947| 1615|
|  19670926| 1436|
|  45006692| 1433|
|   1436172| 1195|
|   2126708| 1122|
|   1436177| 1005|
|  47438714|  978|
|   3855375|  973|
+----------+-----+



In [7]:
# 4. Find the top five neighborhoods with the most listings

listings \
  .groupBy('neighbourhood_cleansed') \
  .count() \
  .orderBy('count', ascending=False) \
  .limit(5) \
  .show()

+----------------------+-----+
|neighbourhood_cleansed|count|
+----------------------+-----+
|           Westminster|11367|
|         Tower Hamlets| 7566|
|                Camden| 6564|
|  Kensington and Ch...| 6348|
|               Hackney| 6279|
+----------------------+-----+



In [8]:
# 5. Get a data frame with the following four columns:
# * Listing's ID
# * Listing's name
# * Reviewer's name
# * Review's comment
# Use "join" to combine data from two datasets

listings.join(reviews, listings.id == reviews.listing_id, 'inner') \
  .select(listings.id, 'name', 'reviewer_name', 'comments') \
  .show(truncate=False)

+-----+-----------------------------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id   |name                               |reviewer_name|comments   

In [9]:
# 6. Get top five listings with the highest average review comment length. Only return listings with at least 5 reviews
# Use the "length" function from the "pyspark.sql.functions" to get a lenght of a review

from pyspark.sql.functions import length, avg, count

reviews_with_comment_length = reviews.withColumn('comment_length', length('comments'))
reviews_with_comment_length \
  .join(listings, reviews_with_comment_length.listing_id == listings.id, 'inner') \
  .groupBy('listing_id').agg(
      avg(reviews_with_comment_length.comment_length).alias('average_comment_length'),
      count(reviews_with_comment_length.id).alias('reviews_count')
  ) \
  .filter('reviews_count >= 5') \
  .orderBy('average_comment_length', ascending=False) \
  .show()

+------------------+----------------------+-------------+
|        listing_id|average_comment_length|reviews_count|
+------------------+----------------------+-------------+
|618608352812465378|    1300.1666666666667|            6|
|          28508447|    1089.3333333333333|            6|
|627425975703032358|     951.7777777777778|            9|
|           2197681|                 939.2|            5|
|          13891813|                 905.0|            5|
|            979753|     893.9230769230769|           13|
|630150178279666225|     890.7272727272727|           11|
|           8856894|     890.1666666666666|            6|
|          29469389|                 885.0|            6|
|          22524075|                 885.0|            5|
|           5555679|     878.7169811320755|          106|
|          33385444|                 848.0|            5|
|            565214|     834.0833333333334|           12|
|          53493254|                 831.0|            7|
|          126

In [13]:
# 7. Using the "join" operator find listings without reviews.
# Hint: Use "left_join" or "left_anti" join type when implementing this

joined_df = listings.join(
    reviews,
    listings.id == reviews.listing_id,
    how='left_outer'
)

joined_df \
  .filter(reviews.listing_id.isNull()) \
  .select('name') \
  .show(truncate=False)

+------------------------------------------------+
|name                                            |
+------------------------------------------------+
|ChiqDoube Room in PrivateAppartment             |
|ROOM TO RENT IN THE OLYMPIC PERIOD              |
|4 bed Beautiful west london home                |
|London, Hoxton. Nice, 2 bedroom, 7th floor flat.|
|Bright Dbl/Nr/ Excellnt Transp                  |
|Stunning Shared Penthouse Apartment             |
|The Old Coach House (Olympics)                  |
|Well furnished room (Olympics site)             |
|Studio 20min Walk from Olympic City             |
|Luxury single room                              |
|Contemporary house London E4                    |
|A lovely one bedroom garden flat!!              |
|Coming to London for the Olympics?              |
|Lovely double room + own bathroom               |
|Double Room close to Olympic Park!              |
|Double bedroom near Olympic Park                |
|SPARE ROOM TO LET DURING OLYMP

In [14]:
# Performing a left anti join to find listings without reviews
listings_without_reviews = listings \
  .join(
    reviews,
    listings.id == reviews.listing_id,
    how='left_anti'
  ) \
  .select('name') \
  .show(truncate=False)

+------------------------------------------------+
|name                                            |
+------------------------------------------------+
|ChiqDoube Room in PrivateAppartment             |
|ROOM TO RENT IN THE OLYMPIC PERIOD              |
|4 bed Beautiful west london home                |
|London, Hoxton. Nice, 2 bedroom, 7th floor flat.|
|Bright Dbl/Nr/ Excellnt Transp                  |
|Stunning Shared Penthouse Apartment             |
|The Old Coach House (Olympics)                  |
|Well furnished room (Olympics site)             |
|Studio 20min Walk from Olympic City             |
|Luxury single room                              |
|Contemporary house London E4                    |
|A lovely one bedroom garden flat!!              |
|Coming to London for the Olympics?              |
|Lovely double room + own bathroom               |
|Double Room close to Olympic Park!              |
|Double bedroom near Olympic Park                |
|SPARE ROOM TO LET DURING OLYMP